# Introduction to Natural Language Processing (NLP) in TensorFlow

### Word Embeddings

Word embeddings, or word vectors, provide a way of mapping words from a vocabulary into a low-dimensional space, where words with similar meanings are close together. Let's play around with a set of pre-trained word vectors, to get used to their properties. There exist many sets of pretrained word embeddings; here, we use ConceptNet Numberbatch, which provides a relatively small download in an easy-to-work-with format (h5).

In [1]:
# Download word vectors
from urllib.request import urlretrieve
import os
if not os.path.isfile('mini.h5'):
    print("Downloading Conceptnet Numberbatch word embeddings...")
    conceptnet_url = 'http://conceptnet.s3.amazonaws.com/precomputed-data/2016/numberbatch/17.06/mini.h5'
    urlretrieve(conceptnet_url, 'mini.h5')

In [0]:
#import urllib2
#response = urllib2.urlopen('https://raw.githubusercontent.com/jsigman/Duke-MLSS-2019/master/movie-simple.txt')
#data = response.read()

In [0]:
#!pip install -q urllib2
#!apt-get -qq install -y urllib2
#import urllib2
###files.download('https://wordpress.org/plugins/about/readme.txt') 
#response = urllib2.urlopen('https://wordpress.org/plugins/about/readme.txt')
#data = response.read()
#files.download('https://raw.githubusercontent.com/jsigman/Duke-MLSS-2019/master/movie-simple.txt')
#from google.colab import files
#files.download('https://github.com/jsigman/Duke-MLSS-2019/blob/master/movie-simple.txt')
#uploaded = files.upload()

To read an `h5` file, we'll need to use the `h5py` package. Below, we use the package to open the `mini.h5` file we just downloaded. We extract from the file a list of utf-8-encoded words, as well as their $300$-dimensional vectors.

In [17]:
# Load the file and pull out words and embeddings
import h5py

with h5py.File('mini.h5', 'r') as f:
    all_words = [word.decode('utf-8') for word in f['mat']['axis1'][:]]
    all_embeddings = f['mat']['block0_values'][:]
    
print("all_words dimensions: {0}".format(len(all_words)))
print("all_embeddings dimensions: {0}".format(all_embeddings.shape))

print(all_words[10000])

all_words dimensions: 362891
all_embeddings dimensions: (362891, 300)
/c/de/lande


Now, `all_words` is a list of $V$ strings (what we call our *vocabulary*), and `all_embeddings` is a $V \times 300$ matrix. The strings are of the form `/c/language_code/word`—for example, `/c/en/cat` and `/c/es/gato`.

We are interested only in the English words. We use Python list comprehensions to pull out the indices of the English words, then extract just the English words (stripping the six-character `/c/en/` prefix) and their embeddings.

In [19]:
# Restrict our vocabulary to just the English words
english_words = [word[6:] for word in all_words if word.startswith('/c/en/')]
english_word_indices = [i for i, word in enumerate(all_words) if word.startswith('/c/en/')]
english_embeddings = all_embeddings[english_word_indices]

print("all_words dimensions: {0}".format(len(english_words)))
print("all_embeddings dimensions: {0}".format(english_embeddings.shape))

print(english_words[10000])

all_words dimensions: 150875
all_embeddings dimensions: (150875, 300)
bajillion


The magnitude of a word vector is less important than its direction; the magnitude can be thought of as representing frequency of use, independent of the semantics of the word. 
Here, we will be interested in semantics, so we *normalize* our vectors, dividing each by its length. 
The result is that all of our word vectors are length 1, and as such, lie on a unit circle. 
The dot product of two vectors is proportional to the cosine of the angle between them, and provides a measure of similarity (the bigger the cosine, the smaller the angle).

<img src="https://github.com/jsigman/Duke-MLSS-2019/blob/master/Figures/cosine_similarity.png?raw=1" alt="cosine" style="width: 500px;"/>
<center>Figure adapted from *[Mastering Machine Learning with Spark 2.x](https://www.safaribooksonline.com/library/view/mastering-machine-learning/9781785283451/ba8bef27-953e-42a4-8180-cea152af8118.xhtml)*</center>

In [0]:
import numpy as np

norms = np.linalg.norm(english_embeddings, axis=1)
normalized_embeddings = english_embeddings.astype('float32') / norms.astype('float32').reshape([-1, 1])

We want to look up words easily, so we create a dictionary that maps us from a word to its index in the word embeddings matrix.

In [0]:
index = {word: i for i, word in enumerate(english_words)}


In [29]:
english_words[10000]

'bajillion'

In [30]:
index['bajillion']

10000

Now we are ready to measure the similarity between pairs of words. We use numpy to take dot products.

In [0]:
def similarity_score(w1, w2):
    score = np.dot(normalized_embeddings[index[w1], :], normalized_embeddings[index[w2], :])
    return score

In [32]:
# A word is as similar with itself as possible:
print('cat\tcat\t', similarity_score('cat', 'cat'))

# Closely related words still get high scores:
print('cat\tfeline\t', similarity_score('cat', 'feline'))
print('cat\tdog\t', similarity_score('cat', 'dog'))

# Unrelated words, not so much
print('cat\tmoo\t', similarity_score('cat', 'moo'))
print('cat\tfreeze\t', similarity_score('cat', 'freeze'))

# Antonyms are still considered related, sometimes more so than synonyms
print('antonyms\topposites\t', similarity_score('antonym', 'opposite'))
print('antonyms\tsynonyms\t', similarity_score('antonym', 'synonym'))

cat	cat	 1.0000001
cat	feline	 0.8199548
cat	dog	 0.590724
cat	moo	 0.0039538303
cat	freeze	 -0.030225191
antonyms	opposites	 0.3941065
antonyms	synonyms	 0.46883982


We can also find, for instance, the most similar words to a given word.

In [0]:
def closest_to_vector(v, n):
    all_scores = np.dot(normalized_embeddings, v)
    best_words = map(lambda i: english_words[i], reversed(np.argsort(all_scores)))
    return [next(best_words) for _ in range(n)]

def most_similar(w, n):
    return closest_to_vector(normalized_embeddings[index[w], :], n)

In [35]:
print(most_similar('cat', 10))
print(most_similar('dog', 10))
print(most_similar('duke', 10))

['cat', 'humane_society', 'kitten', 'feline', 'colocolo', 'cats', 'kitty', 'maine_coon', 'housecat', 'sharp_teeth']
['dog', 'dogs', 'wire_haired_dachshund', 'doggy_paddle', 'lhasa_apso', 'good_friend', 'puppy_dog', 'bichon_frise', 'woof_woof', 'golden_retrievers']
['duke', 'dukes', 'duchess', 'duchesses', 'ducal', 'dukedom', 'duchy', 'voivode', 'princes', 'prince']


**Can you find a polysemous word — a word with multiple meanings — so that the list of the top 10 most related words contains words that aren't themselves related to one another?**

In [36]:
print(most_similar('gusto', 10))
print(most_similar('right', 10))
print(most_similar('cleave', 10))
print(most_similar('ass', 10))
print(most_similar('bad', 10))
print(most_similar('break', 10))

['gusto', 'enthusiasm', 'zest', 'zeal', 'delight', 'fervor', 'relish', 'vigor', 'enthusiasms', 'ardor']
['right', 'righted', 'religious_right', 'rightmost', 'rightward', 'rightwards', 'righting', 'dextral', 'righ', 'rightie']
['cleave', 'cleaved', 'cleaves', 'cleaving', 'scission', 'rend', 'split', 'cleaver', 'divide', 'sever']
['ass', 'asses', 'arse', 'butt', 'butts', 'arses', 'assed', 'azz', 'bunda', 'buttocks']
['bad', 'woe_betide', 'badder', 'baddest', 'from_hell', 'lousy', 'worst', 'riaa', 'unpleasant', 'worse']
['break', 'breaks', 'breaked', 'breaking', 'broke', 'shatter', 'broken', 'breakage', 'shatters', 'smash']


We can also use `closest_to_vector` to find words "nearby" vectors that we create ourselves. This allows us to solve analogies. For example, in order to solve the analogy "man : brother :: woman : ?", we can compute a new vector `brother - man + woman`: the meaning of brother, minus the meaning of man, plus the meaning of woman. We can then ask which words are closest, in the embedding space, to that new vector.

In [38]:
def solve_analogy(a1, b1, a2):
    b2 = normalized_embeddings[index[b1], :] - normalized_embeddings[index[a1], :] + normalized_embeddings[index[a2], :]
    return closest_to_vector(b2, 10)

print(solve_analogy("man", "brother", "woman"))
print(solve_analogy("man", "husband", "woman"))
print(solve_analogy("spain", "madrid", "france"))

['sister', 'brother', 'sisters', 'kid_sister', 'younger_brother', 'niece', 'nieces', 'sistren', 'stepsister', 'daughter']
['wife', 'husband', 'husbands', 'spouse', 'wifes', 'wifey', 'et_ux', 'hubby', 'hotwife', 'wives']
['paris', 'france', 'le_havre', 'in_france', 'montmartre', 'marseille', 'loire_valley', 'saone', 'lyonnais', 'jacques_chirac']


**Note that the new vector, `b2`, is not normalized. Does this matter?  Why or why not?**

These three results are quite good, but in general, the results of these analogies can be disappointing. Try experimenting with other analogies, and see if you can think of ways to get around the problems you notice (i.e., modifications to the solve_analogy algorithm).

### Using word embeddings in deep models
Word embeddings are fun to play around with, but their primary use is that they allow us to think of words as existing in a continuous, Euclidean space; we can then use an existing arsenal of techniques for machine learning with continuous numerical data (like logistic regression or neural networks) to process text.

Let's take a look at an especially simple version of this. We'll perform *sentiment analysis* on a set of movie reviews: in particular, we will attempt to classify a movie review as positive or negative based on its text.

We will use a [Simple Word Embedding Model](http://people.ee.duke.edu/~lcarin/acl2018_swem.pdf) (SWEM, Shen et al. 2018) to do so. We will represent a review as the *mean* of the embeddings of the words in the review. Then we'll train a three-layer MLP (a neural network) to classify the review as positive or negative.

Download the `movie-simple.txt` file from Google Classroom into this directory. Each line of that file contains 

1. the numeral 0 (for negative) or the numeral 1 (for positive), followed by
2. a tab (the whitespace character), and then
3. the review itself.

In [0]:
import string
remove_punct=str.maketrans('','',string.punctuation)

# This function converts a line of our data file into
# a tuple (x, y), where x is 300-dimensional representation
# of the words in a review, and y is its label.
def convert_line_to_example(line):
    # Pull out the first character: that's our label (0 or 1)
    y = int(line[0])
    
    # Split the line into words using Python's split() function
    words = line[2:].translate(remove_punct).lower().split()
    
    # Look up the embeddings of each word, ignoring words not
    # in our pretrained vocabulary.
    embeddings = [normalized_embeddings[index[w]] for w in words
                  if w in index]
    
    # Take the mean of the embeddings
    x = np.mean(np.vstack(embeddings), axis=0)
    return {'x': x, 'y': y}

# Apply the function to each line in the file.
# manually uploaded 'movie-simple.txt' to the File-space (from Table of Contents) from Github
# https://raw.githubusercontent.com/jsigman/Duke-MLSS-2019/master/movie-simple.txt
# sometime soon I'll figure out how to get Python to do this for me.
with open("movie-simple.txt", "r", encoding='utf-8', errors='ignore') as f:
    dataset = [convert_line_to_example(l) for l in f.readlines()]

In [40]:
len(dataset)

1411

Now that we have a dataset, let's shuffle it and do a train/test split. We use a quarter of the dataset for testing, 3/4 for training (but also ensure that we have a whole number of batches in our training set, to make the code nicer later).

In [0]:
import random
random.shuffle(dataset)

batch_size = 100
total_batches = len(dataset) // batch_size
train_batches = 3*total_batches // 4 
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

Time to build our MLP in Tensorflow. We'll use placeholders for `X` and `y` as usual.

In [42]:
import tensorflow as tf

# Placeholders for input
X = tf.placeholder(tf.float32, [None, 300])
y = tf.placeholder(tf.float32, [None, 1])

# Three-layer MLP
h1 = tf.layers.dense(X, 100, tf.nn.relu)
h2 = tf.layers.dense(h1, 20, tf.nn.relu)
logits = tf.layers.dense(h2, 1)
probabilities = tf.sigmoid(logits)

# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(probabilities), y), tf.float32))

# Training
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

# Initialization of variables
init_op = tf.global_variables_initializer()

W0619 20:37:12.442302 139744459282304 deprecation.py:323] From <ipython-input-42-bd1124f959fe>:8: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0619 20:37:12.448512 139744459282304 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0619 20:37:12.767754 139744459282304 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast

We can now begin a session and train our model. We'll train for 250 epochs. When we're finished, we'll evaluate our accuracy on all the test data.

In [43]:
# Train
sess = tf.Session()
sess.run(init_op)

for epoch in range(250):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = [sample['x'] for sample in data]
        labels  = [sample['y'] for sample in data]
        labels = np.array(labels).reshape([-1,1])
        
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
        
    if epoch % 10 == 0:
        print("Epoch: {0} \t Loss: {1} \t Acc: {2}".format(epoch, l, acc))
    
    random.shuffle(train)
        
# Evaluate on test set
test_reviews = [sample['x'] for sample in test]
test_labels  = [sample['y'] for sample in test]
test_labels  = np.array(test_labels).reshape([-1, 1])

acc = sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
print("Final accuracy: {0}".format(acc))

Epoch: 0 	 Loss: 0.6946114301681519 	 Acc: 0.46000000834465027
Epoch: 10 	 Loss: 0.6793479323387146 	 Acc: 0.5600000023841858
Epoch: 20 	 Loss: 0.6740855574607849 	 Acc: 0.5799999833106995
Epoch: 30 	 Loss: 0.6611305475234985 	 Acc: 0.6299999952316284
Epoch: 40 	 Loss: 0.6463670134544373 	 Acc: 0.6299999952316284
Epoch: 50 	 Loss: 0.6332020163536072 	 Acc: 0.6700000166893005
Epoch: 60 	 Loss: 0.600393533706665 	 Acc: 0.7400000095367432
Epoch: 70 	 Loss: 0.5740084648132324 	 Acc: 0.8100000023841858
Epoch: 80 	 Loss: 0.5156574845314026 	 Acc: 0.8299999833106995
Epoch: 90 	 Loss: 0.44840899109840393 	 Acc: 0.8899999856948853
Epoch: 100 	 Loss: 0.3735537827014923 	 Acc: 0.8799999952316284
Epoch: 110 	 Loss: 0.3248090445995331 	 Acc: 0.9100000262260437
Epoch: 120 	 Loss: 0.2852449119091034 	 Acc: 0.9300000071525574
Epoch: 130 	 Loss: 0.20646663010120392 	 Acc: 0.949999988079071
Epoch: 140 	 Loss: 0.20869185030460358 	 Acc: 0.9399999976158142
Epoch: 150 	 Loss: 0.19813886284828186 	 Acc: 0.9

We can now examine what our model has learned, seeing how it responds to word vectors for different words:

In [44]:
# Check some words
words_to_test = ["exciting", "hated", "boring", "loved", "extremely", "rather", "quite"]

for word in words_to_test:
    print(word, sess.run(probabilities, feed_dict={X: normalized_embeddings[index[word]].reshape(1, 300)}))

exciting [[0.9999918]]
hated [[9.049819e-07]]
boring [[9.248749e-06]]
loved [[1.]]
extremely [[0.504851]]
rather [[0.11691659]]
quite [[0.9281063]]


Try some words of your own!

In [0]:
sess.close()
tf.reset_default_graph()

This model works great for such a simple dataset, but does a little less well on something more complex. `movie-pang02.txt`, for instance, has 2000 longer, more complex movie reviews. It's in the same format as our simple dataset. On those longer reviews, this model achieves only 60-80% accuracy. (Increasing the number of epochs to, say, 1000, does help.)